In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy, spearmanr, wasserstein_distance
from sklearn.metrics import roc_auc_score

# Load merged results
df = pd.read_csv("merged_scores.csv")
# Example: each row = one candidate–JD pair or one candidate–main JD pair


In [ ]:
def mean_shift(df, col_ref, col_comp):
    return (df[col_comp] - df[col_ref]).mean()

def var_ratio(df, col_ref, col_comp):
    var_ref = df[col_ref].var()
    var_comp = df[col_comp].var()
    return var_comp / var_ref if var_ref != 0 else np.nan

print("Unfair vs Baseline – mean shift:",
      mean_shift(df, "baseline_score", "unfair_score"))
print("ITI vs Baseline – mean shift:",
      mean_shift(df, "baseline_score", "iti_fair_score"))

print("Unfair vs Baseline – variance ratio:",
      var_ratio(df, "baseline_score", "unfair_score"))
print("ITI vs Baseline – variance ratio:",
      var_ratio(df, "baseline_score", "iti_fair_score"))


In [ ]:
def bias_gap(df, group_col, score_col):
    group_means = df.groupby(group_col)[score_col].mean().dropna()
    if len(group_means) < 2:
        return np.nan
    # For binary group (e.g. Male/Female)
    # take max minus min
    return group_means.max() - group_means.min()

for col in ["baseline_score", "unfair_score", "iti_fair_score"]:
    print(f"BiasGap gender, {col}:",
          bias_gap(df, "gender", col))
    print(f"BiasGap race, {col}:",
          bias_gap(df, "race", col))


In [ ]:
def score_hist_probs(series, bins=20, eps=1e-9):
    counts, bin_edges = np.histogram(series.dropna(), bins=bins, range=(0, 100), density=False)
    probs = counts.astype(float)
    probs = probs / probs.sum() if probs.sum() > 0 else probs
    # avoid 0 for KL
    probs = np.clip(probs, eps, 1.0)
    return probs, bin_edges

def kl_divergence(ref, comp):
    p, _ = score_hist_probs(ref)
    q, _ = score_hist_probs(comp)
    return entropy(p, q)

def emd(ref, comp):
    # Wasserstein distance over score values
    return wasserstein_distance(ref.dropna(), comp.dropna())

def spearman_ref(ref, comp):
    return spearmanr(ref, comp, nan_policy="omit").correlation

print("KL(unfair || baseline):", kl_divergence(df["baseline_score"], df["unfair_score"]))
print("KL(iti || baseline):", kl_divergence(df["baseline_score"], df["iti_fair_score"]))

print("EMD(unfair, baseline):", emd(df["baseline_score"], df["unfair_score"]))
print("EMD(iti, baseline):", emd(df["baseline_score"], df["iti_fair_score"]))

print("Spearman(baseline, unfair):", spearman_ref(df["baseline_score"], df["unfair_score"]))
print("Spearman(baseline, iti):", spearman_ref(df["baseline_score"], df["iti_fair_score"]))


In [ ]:
plt.figure()
plt.hist(df["baseline_score"].dropna(), bins=20, alpha=0.5, label="Baseline")
plt.hist(df["unfair_score"].dropna(), bins=20, alpha=0.5, label="Unfair")
plt.hist(df["iti_fair_score"].dropna(), bins=20, alpha=0.5, label="ITI Fair")
plt.xlabel("Score")
plt.ylabel("Count")
plt.title("Score Distributions – Baseline vs Unfair vs ITI")
plt.legend()
plt.show()


In [ ]:
plt.figure()
for gender, sub in df.groupby("gender"):
    plt.hist(sub["unfair_score"].dropna(), bins=20, alpha=0.4, label=f"Unfair – {gender}")
plt.xlabel("Unfair Score")
plt.ylabel("Count")
plt.title("Unfair Model Scores by Gender")
plt.legend()
plt.show()

plt.figure()
for gender, sub in df.groupby("gender"):
    plt.hist(sub["iti_fair_score"].dropna(), bins=20, alpha=0.4, label=f"ITI – {gender}")
plt.xlabel("ITI Fair Score")
plt.ylabel("Count")
plt.title("ITI Model Scores by Gender")
plt.legend()
plt.show()


In [ ]:
def top1_accuracy(df_multi, score_col):
    # argmax JD per candidate
    idx = df_multi.groupby("candidate_id")[score_col].idxmax()
    pred = df_multi.loc[idx, ["candidate_id", "jd_industry", "true_industry"]]
    return (pred["jd_industry"] == pred["true_industry"]).mean()

for col in ["baseline_score", "unfair_score", "iti_fair_score"]:
    acc = top1_accuracy(df_multi, col)
    print(f"Top-1 accuracy ({col}):", acc)


In [ ]:
def industry_auc(df_multi, score_col):
    aucs = []
    for cid, sub in df_multi.groupby("candidate_id"):
        y_true = (sub["jd_industry"] == sub["true_industry"]).astype(int)
        if y_true.nunique() < 2:
            continue
        try:
            auc = roc_auc_score(y_true, sub[score_col])
            aucs.append(auc)
        except ValueError:
            continue
    return np.mean(aucs) if aucs else np.nan

for col in ["baseline_score", "unfair_score", "iti_fair_score"]:
    auc = industry_auc(df_multi, col)
    print(f"Industry AUC ({col}):", auc)
